In [30]:
# import torch
import numpy as np
from scipy.spatial.transform import Rotation as R

def compute_global_rotation(pose_axis_anges, joint_idx):
    """
    calculating joints' global rotation
    Args:
        pose_axis_anges (np.array): SMPLX's local pose (22,3)
    Returns:
        np.array: (3, 3)
    """
    global_rotation = np.eye(3)
    parents = [-1,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  9,  9, 12, 13, 14, 16, 17, 18, 19]
    while joint_idx != -1:
        joint_rotation = R.from_rotvec(pose_axis_anges[joint_idx]).as_matrix()
        global_rotation = joint_rotation @ global_rotation
        joint_idx = parents[joint_idx]
    return global_rotation

M = np.diag([-1, 1, 1])     

# GVHMR data

In [1]:
import pickle
file_gvhmr = r"D:\AI\hamer\sample_gvhmr_and_hamer\gvhmr_test1_person0.pkl"
with(open(file_gvhmr, "rb")) as f:
    pkl_gvhmr = pickle.load(f)


In [41]:
# pkl_gvhmr.keys()
pkl_gvhmr["smpl_params_global"].keys()

dict_keys(['body_pose', 'global_orient', 'transl', 'betas'])

In [42]:

gvhmr_smplx_params = {}
gvhmr_smplx_params["global_orient"] = pkl_gvhmr["smpl_params_global"]["global_orient"]
gvhmr_smplx_params["body_pose"] = pkl_gvhmr["smpl_params_global"]["body_pose"]
gvhmr_smplx_params["transl"] = pkl_gvhmr["smpl_params_global"]["transl"]

In [6]:
gvhmr_smplx_params["global_orient"].shape

(11, 3)

# Hamer data

In [44]:
file_gvhmr = r"D:\AI\hamer\sample_gvhmr_and_hamer\hamer_result_test1.pkl"
with(open(file_gvhmr, "rb")) as f:
    pkl_hamer = pickle.load(f)

In [45]:
pkl_hamer.keys()

dict_keys(['pred_cam', 'global_orient', 'hand_pose', 'betas', 'pred_cam_t', 'focal_length', 'pred_keypoints_3d', 'pred_vertices', 'pred_keypoints_2d'])

In [46]:
hamer_mano_params = {}
hamer_mano_params["global_orient"] = pkl_hamer["global_orient"]
hamer_mano_params["hand_pose"] = pkl_hamer["hand_pose"]

# code to convert

In [47]:
import numpy as np

M = np.diag([-1, 1, 1])                                                                                                              # Preparing for the left hand switch

# Assuming that your data are stored in gvhmr_smplx_params and hamer_mano_params
# full_body_pose = torch.concatenate((gvhmr_smplx_params["global_orient"][0], gvhmr_smplx_params["body_pose"][0].reshape(21, 3)), dim=0)     # gvhmr_smplx_params["global_orient"]: (3, 3)
full_body_pose = np.concatenate((gvhmr_smplx_params["global_orient"][0].reshape(-1,3), gvhmr_smplx_params["body_pose"][0].reshape(21, 3)))     # gvhmr_smplx_params["global_orient"]: (3, 3)
left_elbow_global_rot = compute_global_rotation(full_body_pose, 18) # left elbow IDX: 18
right_elbow_global_rot = compute_global_rotation(full_body_pose, 19) # left elbow IDX: 19

left_wrist_global_rot = hamer_mano_params["global_orient"][0]                                                          # hamer_mano_params["global_orient"]: (2, 3, 3)
left_wrist_global_rot = M @ left_wrist_global_rot @ M                                                                                # mirror switch
left_wrist_pose = np.linalg.inv(left_elbow_global_rot) @ left_wrist_global_rot

right_wrist_global_rot = hamer_mano_params["global_orient"][1]
right_wrist_pose = np.linalg.inv(right_elbow_global_rot) @ right_wrist_global_rot

left_wrist_pose_vec = R.from_matrix(left_wrist_pose).as_rotvec()
right_wrist_pose_vec = R.from_matrix(right_wrist_pose).as_rotvec()

left_hand_pose = np.ones(45)
right_hand_pose = np.ones(45)
for i in range(15):
    left_finger_pose = M @ hamer_mano_params["hand_pose"][0][i] @ M                                                    # hamer_mano_params["hand_pose"]: (2, 15, 3, 3)
    left_finger_pose_vec = R.from_matrix(left_finger_pose).as_rotvec()
    left_hand_pose[i*3: i*3+3] = left_finger_pose_vec
    
    right_finger_pose = hamer_mano_params["hand_pose"][1][i]
    right_finger_pose_vec = R.from_matrix(right_finger_pose).as_rotvec()
    right_hand_pose[i*3: i*3+3] = right_finger_pose_vec

gvhmr_smplx_params["body_pose"][0][57: 60] = left_wrist_pose_vec
gvhmr_smplx_params["body_pose"][0][60: 63] = right_wrist_pose_vec
gvhmr_smplx_params["left_hand_pose"] = left_hand_pose
gvhmr_smplx_params["right_hand_pose"] = right_hand_pose

In [48]:
pkl_out = r"D:\AI\hamer\sample_gvhmr_and_hamer\gvhmr_with_hammer.pkl"
with(open(pkl_out, "wb")) as f:
    pickle.dump(gvhmr_smplx_params, f)


In [49]:
gvhmr_smplx_params.keys()


dict_keys(['global_orient', 'body_pose', 'transl', 'left_hand_pose', 'right_hand_pose'])